In [1]:
import pprint
import time
from scraper.wiki.heroes import get_heroes_list, get_hero_abilities
    

In [2]:
heroes_data = {}
abilities_data = {}

heroes_list = get_heroes_list()
time.sleep(1)

for hero_name, hero_url in heroes_list:
    hero_data = {
        "name": hero_name,
        "url": hero_url,
        # TODO: Health?
    }
    
    hero_abilities = get_hero_abilities(hero_url)
    hero_data["abilities"] = [ability["name"] for ability in hero_abilities]
    for ability in hero_abilities:
        abilities_data[ability["name"]] = ability
    
    heroes_data[hero_name] = hero_data
    time.sleep(1)
    break

In [3]:
heroes_data

{'Ana': {'name': 'Ana',
  'url': 'https://overwatch.fandom.com/wiki/Ana',
  'abilities': ['Role: Support',
   'Biotic Rifle',
   'Sleep Dart',
   'Biotic Grenade',
   'Nano Boost']}}

In [4]:
abilities_data

{'Role: Support': {'image_url': 'https://static.wikia.nocookie.net/overwatch_gamepedia/images/5/5f/SupportIcon.png/revision/latest/scale-to-width-down/100?cb=20151109212032',
  'summary': 'Begin regenerating health more quickly.',
  'name': 'Role: Support'},
 'Biotic Rifle': {'image_url': 'https://static.wikia.nocookie.net/overwatch_gamepedia/images/f/fc/Ability-ana1.png/revision/latest/scale-to-width-down/100?cb=20181119020608',
  'affects': ['Blocked by barriers.',
   'Blocked by Defense Matrix/Kinetic Grasp.',
   'Blocked by Deflect/Javelin Spin.',
   'Affected by damage boosts.',
   'Affected by Amplification Matrix.'],
  'summary': 'Long-range rifle that heals allies and damages enemies. Hold to zoom in.',
  'name': 'Biotic Rifle',
  'keybind': 'Key mouse 1',
  'Damage': '70 over 0.59 seconds',
  'Headshot': '✕',
  'Healing': '70 over 0.59 seconds',
  'Rate of fire': '0.8 seconds recovery',
  'Ammo': '15',
  'Reload time': '1.5 seconds animation',
  'Cast time': '0.24 seconds for 